In [23]:
%load_ext pycodestyle_magic

%pycodestyle_off

In [18]:
#%load_ext pycodestyle_magic
#%pycodestyle_on

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime
import numpy as np
import folium
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mtick
import pickle



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,\
 OneHotEncoder, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

import time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
# define categorical, numerical and date columns
CAT_COLS = ["dia_semana", "codigo_cierre", "año_cierre", "mes_cierre", "mes", "delegacion_inicio",
            "incidente_c4", "clas_con_f_alarma", "tipo_entrada", "delegacion_cierre", "hora_creacion",
           "hora_cierre"]

DATE_COLS = ["fecha_creacion", "fecha_cierre"]

NUM_COLS = ["latitud", "longitud"]#esto irá en el script ingestion .py
def ingest_file(file_name):
    """
    Function to retrieve and return the accidents dataset.
    Parameters:
    -----------
    file_name: str
               Path to the file.
    Returns:
    --------
    df: pandas dataframe
    """
    df = pd.read_csv(file_name)
    return df


def drop_cols(df):
    """
    Function to drop unnnecesary columns in the dataset.
    """
    df.drop(columns = ['folio', 'geopoint', 'mes', 'mes_cierre', 'hora_cierre', 'año_cierre'], inplace = True)
    return df



def fill_na(df):
    """
    Function to fill null values in a dataframe.
    """
    #aquí podemos ir agregando más cosas cuando descubramos 
    #cómo imputar valores faltantes para latitud y longitud
    df.fillna({
        'delegacion_inicio': 'No Disponible',
        'delegacion_cierre': 'No Disponible'
              }, inplace = True)
    return df


def categoric_transformation(col,df):
    df[col] = df[col].astype("category")
    return df 

def create_categorical(cols, df):
    """
    Function to transform and prepare the categorical features in the dataset.
    """
    #transform to appropriate data type
    for col in cols: 
        df = categoric_transformation(col, df)
     
    return df


def date_transformation(col,df):
    """
    Function to prepare and transform date-type columns. 
    """
    df[col] = pd.to_datetime(df[col], dayfirst=True)
    return df

def create_date_cols(cols, df):
    for col in cols:
        df = date_transformation(col, df)
    return df 



def generate_label(df):
    """
    Function to create a new column indicating whether there was
    a false alarm or not. 
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    --------
    df: pandas dataframe
    """
    #transformamos la columna para solo quedarnos con la letra del código
    df["codigo_cierre"] = df["codigo_cierre"].apply(lambda x: x[1])
    df['label'] = np.where(
        (df.codigo_cierre == 'F') | (df.codigo_cierre == 'N'), 1, 0)
    return df 


def clean_hora_creacion(df):
    """
    Function to transform hours with incorrect format to timedelta format. 
    """
    horas_raw = df.hora_creacion.values.tolist()
    horas_clean = [datetime.timedelta(days=float(e)) if e.startswith("0.") else e for e in horas_raw]
    df["hora_creacion"] = horas_clean
    return df 


def create_simple_hour(df):
    """
    Function to extract the hour from the column "hora_creacion"
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with a new column indicating the hour. 
    """
    #la función se podria adaptar para devolver minuto o segundo pero no lo considero necesario
    pattern = '\d+' #encuentra uno o más dígitos
    horas_raw = df.hora_creacion.astype(str).values #son así: '22:35:04', '22:50:49', '09:40:11'
    n = len(horas_raw)
    horas_clean = [0]*n #es más rápido reasignar valores que hacer .append()
    for i in range(n):
        hora_raw = horas_raw[i]
        hora_clean = re.match(pattern, hora_raw)[0] #solo queremos la hora, esto devuelve un objeto
        horas_clean[i] = hora_clean
    
    df["hora_simple"] = horas_clean
    return df 


def add_date_columns(df):
    """
    Esta función es muy importante puesto que nos ayudará a crear el mes, día y año de creación
    del registro. De esta manera podemos prescindir de las fechas de cierre, que no tendríamos en tiempo
    real en un modelo. 
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with 4 new columns
    """
    mapping_meses = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo",
                       6: "Junio", 7: "Julio", 8: "Agosto", 9: "Septiembre", 10: "Octubre",
                       11: "Noviembre", 12: "Diciembre"}
    
    
    df["año_creacion"] = df.fecha_creacion.dt.year
    df["mes_creacion"] = df.fecha_creacion.dt.month
    df["dia_creacion"] = df.fecha_creacion.dt.day
    df["mes_creacion_str"] = df.mes_creacion.map(mapping_meses)
    df["año_creacion"] = df["año_creacion"].astype(str)
    return df 


def create_time_blocks(df):
    """
    Function to group the hour of the day into 3-hour blocks.
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with a new column indicating the time-block.
    """
    horas_int = set(df.hora_simple.astype(int).values) #estaba como categórico
    f = lambda x: 12 if x == 0 else x
    mapping_hours = {}
    for hora in horas_int:
        grupo = (hora // 3) * 3
        if grupo < 12: 
            nombre_grupo = str(f(grupo)) + "-" + str(grupo + 2) + " a.m."
        else:
            hora_tarde = grupo % 12
            nombre_grupo = str(f(hora_tarde)) + "-" + str(hora_tarde + 2) + " p.m."
        mapping_hours[hora] = nombre_grupo
    
    df["espacio_del_dia"] = df["hora_simple"].astype(int).map(mapping_hours)
    return df
    

    

    
def basic_preprocessing(path):
    """
    Function to summarize all the preprocessing done to the data.
    Parameters:
    -----------
    path: str
          Path to your file
    
    Returns:
    ---------
    df: pandas dataframe
    """
    df = ingest_file(path) 
    df = generate_label(df)
    df = fill_na(df) 
    df = clean_hora_creacion(df)
    df = create_categorical(CAT_COLS, df) #transform to appropriate data types
    df = create_date_cols(DATE_COLS, df)
    df = add_date_columns(df)
    df = create_simple_hour(df)
    df = create_time_blocks(df)
    df = drop_cols(df)
    
    return df

In [3]:
incidentes_viales_df = basic_preprocessing('../data/incidentes-viales-c5.csv')

In [4]:
 incidentes = incidentes_viales_df.drop(['latitud', 'longitud', 'codigo_cierre',
                                           'fecha_creacion', 'fecha_cierre',
                                           'hora_creacion', 'clas_con_f_alarma',
                                           'año_creacion', 'dia_creacion', 'mes_creacion',
                                           'delegacion_cierre'
                                           ], axis=1)

In [5]:
incidentes

,dia_semana,delegacion_inicio,incidente_c4,tipo_entrada,label,mes_creacion_str,hora_simple,espacio_del_dia
0,Sábado,VENUSTIANO CARRANZA,accidente-choque sin lesionados,BOTÓN DE AUXILIO,0,Enero,22,9-11 p.m.
1,Sábado,CUAJIMALPA,accidente-choque con lesionados,BOTÓN DE AUXILIO,0,Enero,22,9-11 p.m.
2,Domingo,TLALPAN,accidente-choque sin lesionados,LLAMADA DEL 066,1,Enero,09,9-11 a.m.
3,Domingo,MAGDALENA CONTRERAS,accidente-choque sin lesionados,LLAMADA DEL 066,1,Enero,22,9-11 p.m.
4,Domingo,MIGUEL HIDALGO,accidente-choque sin lesionados,LLAMADA DEL 066,0,Enero,04,3-5 a.m.
...,...,...,...,...,...,...,...,...
1383133,Sábado,GUSTAVO A. MADERO,lesionado-atropellado,LLAMADA DEL 911,0,Octubre,11,9-11 a.m.
1383134,Sábado,CUAUHTEMOC,accidente-ciclista,LLAMADA DEL 911,0,Octubre,11,9-11 a.m.
1383135,Sábado,COYOACAN,accidente-choque con lesionados,LLAMADA DEL 911,0,Octubre,12,12-2 p.m.
1383136,Sábado,IZTAPALAPA,accidente-choque sin lesionados,LLAMADA DEL 911,1,Octubre,15,3-5 p.m.


Transformamos variables categóricas con OneHotEncoder

In [6]:
transformers_2 = [('one_hot', OneHotEncoder(), ['dia_semana', 'delegacion_inicio', 'incidente_c4',
                                               'tipo_entrada', 'espacio_del_dia', 'mes_creacion_str', 
                                                'hora_simple'])]

col_trans_2 = ColumnTransformer(transformers_2, remainder="drop", n_jobs=-1, verbose=True)

In [7]:
col_trans_2.fit(incidentes)

ColumnTransformer(n_jobs=-1,
                  transformers=[('one_hot', OneHotEncoder(),
                                 ['dia_semana', 'delegacion_inicio',
                                  'incidente_c4', 'tipo_entrada',
                                  'espacio_del_dia', 'mes_creacion_str',
                                  'hora_simple'])],
                  verbose=True)

In [8]:
incidente_input_vars = col_trans_2.transform(incidentes)

In [9]:
incidente_input_vars[0:5,:]

<5x113 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [10]:
final_df = pd.DataFrame(incidente_input_vars)
final_df

,0
0,"(0, 5)\t1.0\n (0, 22)\t1.0\n (0, 27)\t1.0\..."
1,"(0, 5)\t1.0\n (0, 11)\t1.0\n (0, 25)\t1.0\..."
2,"(0, 0)\t1.0\n (0, 21)\t1.0\n (0, 27)\t1.0\..."
3,"(0, 0)\t1.0\n (0, 16)\t1.0\n (0, 27)\t1.0\..."
4,"(0, 0)\t1.0\n (0, 17)\t1.0\n (0, 27)\t1.0\..."
...,...
1383133,"(0, 5)\t1.0\n (0, 13)\t1.0\n (0, 43)\t1.0\..."
1383134,"(0, 5)\t1.0\n (0, 12)\t1.0\n (0, 28)\t1.0\..."
1383135,"(0, 5)\t1.0\n (0, 10)\t1.0\n (0, 25)\t1.0\..."
1383136,"(0, 5)\t1.0\n (0, 15)\t1.0\n (0, 27)\t1.0\..."


In [ ]:
# Transformaciones ciclicas?
# HOURS = 24

#final_df['sin_hr'] = np.sin(2*np.pi*final_df.hour/HOURS)
#final_df['cos_hr'] = np.cos(2*np.pi*final_df.hour/HOURS)

In [11]:
# Se eliminarán los features con menos del 7%
variance_threshold = VarianceThreshold(threshold=0.07)
variance_threshold.fit(incidente_input_vars)

VarianceThreshold(threshold=0.1)

In [12]:
variance_threshold.transform(incidente_input_vars)

<1383138x18 sparse matrix of type '<class 'numpy.float64'>'
	with 5189951 stored elements in Compressed Sparse Row format>

In [47]:
variance_threshold.variances_

array([1.10624662e-01, 1.22735876e-01, 1.14508829e-01, 1.18324988e-01,
       1.20513290e-01, 1.30849058e-01, 1.38482678e-01, 6.83507347e-02,
       4.20408739e-02, 6.97353427e-02, 7.32931485e-02, 1.89410199e-02,
       9.17748871e-02, 9.36266865e-02, 4.05505916e-02, 1.35569539e-01,
       1.42943125e-02, 7.67517022e-02, 5.02532276e-03, 1.14219949e-04,
       2.35786241e-02, 5.91581834e-02, 6.35880865e-02, 3.65665256e-02,
       2.16897626e-06, 1.75888331e-01, 1.68461965e-03, 2.47515873e-01,
       1.96123515e-03, 3.97630699e-05, 2.24123010e-05, 3.86131149e-02,
       1.77900320e-03, 4.03739193e-03, 8.09819018e-04, 3.54141387e-04,
       6.15611151e-04, 2.24391510e-02, 7.82389038e-04, 1.42587354e-03,
       1.10605793e-04, 5.07283948e-04, 4.35784544e-03, 1.20355062e-01,
       3.97630699e-05, 7.22993134e-07, 2.68881304e-04, 1.44598522e-06,
       3.90264149e-04, 1.66285776e-05, 5.85590566e-05, 5.36171834e-02,
       2.36291730e-03, 6.80914207e-03, 2.22794267e-01, 2.48807168e-01,
      

## Features importantes

In [13]:
X = incidente_input_vars
y = incidentes.label.values.reshape(incidente_input_vars.shape[0],)

In [14]:
print(X.shape, y.shape)

(1383138, 113) (1383138,)


In [15]:
np.random.seed(1993)

In [ ]:
# Creamos el modelo 
clf = RandomForestClassifier(n_estimators=100)

# Separamos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Entrenamos
clf.fit(X_train, y_train)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_).sort_values(ascending=False)
feature_imp

## Model selection

In [16]:
# ocuparemos un RF
classifier = RandomForestClassifier(oob_score=True, random_state=1234)
# separando en train, test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# definicion de los hiperparametros que queremos probar
hyper_param_grid = {'n_estimators': [100], 
                    'max_depth': [8],
                    'min_samples_split': [5]}

# ocupemos grid search!
gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision',
                           cv = 5, 
                           n_jobs = -1)
start_time = time.time()
gs.fit(X, y)
print("Tiempo en ejecutar: ", time.time() - start_time)

KeyboardInterrupt: 

In [ ]:
# de los valores posibles que pusimos en el grid, cuáles fueron los mejores
gs.best_params_

In [ ]:
# mejor score asociado a los modelos generados con los diferentes hiperparametros
# corresponde al promedio de los scores generados con los cv
gs.best_score_

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_estimator_.oob_score_

### Pruebas

In [ ]:
predicted_labels = gs.predict(X_test)

In [ ]:
predicted_scores = gs.predict_proba(X_test)

## Pickle

In [ ]:
# Guardar en el pickle
pickle.dump(incidentes, open(r"\output\fe_df.pkl", "wb"))

In [ ]:
# Recuperar el pickle
incidentes_pkl = pickle.load(open(r"\output\fe_df.pkl", "rb"))

In [ ]:
incidentes_pkl.head()

In [ ]:
"""
Módulo feature engineering.
Nuestro target es...
Tenemos un problema priorización de recursos? Creo que prevención.
"""

def load_transformation(path):
    """
    Cargar pickle que se generó durante la transformación
    :param path: Path donde se encuentra el pickle
    :return:
    """
    output_path = os.path.join(path, "transformation_df.pkl")
    
    # Recuperar el pickle
    incidentes_pkl = pickle.load(open(output_path, "rb"))
    
    return incidentes_pkl


def feature_generation(df):
    """
    Crear nuevos features útiles.

    :param df: Dataframe del cual se generarán nuevas variables
    :return:
    """

    return df


def feature_selection(df):
    """
    Seleccionaremos las variables importantes
    :param df: Dataframe del que se seleccionarán variables.
    :return:
    """

def save_fe(df, path):
    """
    Guardar el dataframe en un pickle
    :param df: Dataframe que ya tiene los features que se ocuparán.
    :param path:
    :return:
    """
    output_path = os.path.join(path, "output", "fe_df.pkl")
    # Guardar en el pickle
    pickle.dump(df, open(output_path, "wb"))